# 'Titanic' challenge of Kaggle

The 'Titanic' data challenge from Kaggle, done on my own. I'll attempt to use multiple linear regression as described in 'Data science from scratch'. 

Description of the challenge from Kaggle: "One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

In this challenge, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy."

NB: Another notebook on linear regression with Python: http://www.dataschool.io/linear-regression-in-python/


**To do**
* See how the model behaves if different parameters are taken into account (only one column, etc.). (The model seems to be almost as good if only the 'Sex' column is taken into account, which is a bit worriesome...).
* See what I can learn about the significance of the different parameters if I look at standard errors of the fits.
* See how good predictions are compared to other predictions on the leaderboard.
* Think: Why would linear regression actually make sense?
* Can one extract how much each parameter contributes to the survival? Would need some sort of normalisation, since some values (e.g. 'Age') are much larger than others (e.g. 'Sex')


### Import stuff:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

### Import data set and have a first look at it:

In [2]:
titanic_train_original = pd.read_csv('train.csv')
titanic_test_original = pd.read_csv('test.csv')
print ('Training data set:')
print (titanic_train_original.head())
print (titanic_train_original.describe())
print ('Testing data set:')
print (titanic_test_original.head())
print (titanic_test_original.describe())

Training data set:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex  Age  SibSp  \
0                            Braund, Mr. Owen Harris    male   22      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female   38      1   
2                             Heikkinen, Miss. Laina  female   26      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female   35      1   
4                           Allen, Mr. William Henry    male   35      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN 

There is one more column in 'titanic_train' than in 'titanic_test', namely the 'survived' column. This is the case because this is what the model needs to predict and what one needs to submit to kaggle. 

Some of the columns are not numeric data, namely 'Name', 'Sex', 'Ticket', 'Cabin' and 'Embarked'. Some of these columns are probabily worth ignoring, since it will be hard to obtain meaningful information from them. The once which might be useful - for the moment we will use 'Sex' and 'Embarked' - need to be transformed to numerical data to implement the fit. To replace all values with numbers, first determine which values exist in the respective columns.

In [3]:
print (titanic_train_original['Sex'].unique())
print (titanic_train_original['Embarked'].unique())

['male' 'female']
['S' 'C' 'Q' nan]


Use the following replacement rules:

In 'Sex', replace 'male' => 0 and 'female' => 1. 
In 'Embarked', replace 'S' => 0, 'C' => 1 and 'Q' => 2.

To leave the original data frames intact, I will make copies of the two data sets for further manipulation.

In [4]:
# Make copies of original data frames for further manipulation
titanic_train = titanic_train_original.copy()
titanic_test = titanic_test_original.copy()

titanic_train['Sex'].replace(['male','female'],[0,1],inplace=True)
titanic_test['Sex'].replace(['male','female'],[0,1],inplace=True)
titanic_train['Embarked'].replace(['S','C','Q'],[0,1,2],inplace=True)
titanic_test['Embarked'].replace(['S','C','Q'],[0,1,2],inplace=True)

print (titanic_train.describe())
print (titanic_test.describe())

       PassengerId    Survived      Pclass         Sex         Age  \
count   891.000000  891.000000  891.000000  891.000000  714.000000   
mean    446.000000    0.383838    2.308642    0.352413   29.699118   
std     257.353842    0.486592    0.836071    0.477990   14.526497   
min       1.000000    0.000000    1.000000    0.000000    0.420000   
25%     223.500000    0.000000    2.000000    0.000000   20.125000   
50%     446.000000    0.000000    3.000000    0.000000   28.000000   
75%     668.500000    1.000000    3.000000    1.000000   38.000000   
max     891.000000    1.000000    3.000000    1.000000   80.000000   

            SibSp       Parch        Fare    Embarked  
count  891.000000  891.000000  891.000000  889.000000  
mean     0.523008    0.381594   32.204208    0.362205  
std      1.102743    0.806057   49.693429    0.636157  
min      0.000000    0.000000    0.000000    0.000000  
25%      0.000000    0.000000    7.910400    0.000000  
50%      0.000000    0.000000   1

There are values missing (i.e. NA, null or NaN) in several columns (wherever 'count' is smaller than 891 in the training data and smaller than 418 in the testing data). To further treat the data, I will fill in some values for those (since we don't want to throw away the corresponding rows). An accepted way of doing so is to replace the missing values with the median of the corresponding column in the case of numerical data. For the non-numerical columns, I will use the most frequent value of the column.

Note that for the test data set, I will use the median of the training data to replace fitting values. Not sure why this is done, but apparently it is the way.

In [5]:
# replace missing values in numerical columns with median of column
titanic_train['Age'].fillna(titanic_train['Age'].median(),inplace=True)
titanic_test['Age'].fillna(titanic_train['Age'].median(),inplace=True)
titanic_test['Fare'].fillna(titanic_train['Fare'].median(),inplace=True)

# replace missing values of train['Embarked']
print(titanic_train['Embarked'].value_counts())
titanic_train['Embarked'].fillna(0,inplace=True)

0    644
1    168
2     77
Name: Embarked, dtype: int64


### Linear regression using numpy.linalg.lstsq

From the Numpy documentation: "numpy.linalg.lstsq(a, b, rcond=-1): Return the least-squares solution to a linear matrix equation.

Solves the equation a x = b by computing a vector x that minimizes the Euclidean 2-norm || b - a x ||^2. The equation may be under-, well-, or over- determined (i.e., the number of linearly independent rows of a can be less than, equal to, or greater than its number of linearly independent columns). If a is square and of full rank, then x (but for round-off error) is the “exact” solution of the equation."

To use this, I probably need to create numpy arrays for a (all the parameters) and b (survived column). The resulting x would give the parameters of the prediction model. 

In [6]:
print (titanic_train.columns[:])
a = titanic_train.as_matrix(columns=['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

# add a first column of '1' for the constant of the linear fit
a = np.concatenate((np.ones((891,1)),a),axis=1)

b = titanic_train.as_matrix(columns=['Survived'])

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


Attempt to use the least square fit:

In [7]:
beta_np, _, _ ,_ = np.linalg.lstsq(a,b)
print (beta_np)

[[  7.86246139e-01]
 [ -1.75625202e-01]
 [  5.04721091e-01]
 [ -5.85421870e-03]
 [ -4.21320820e-02]
 [ -1.51876675e-02]
 [  3.30107827e-04]
 [  3.94775733e-02]]


Like this, it is a bit hard to tell if this is doing good or not, because I cannot test my prediction algorithm on anything (since I have used all my training data for the fit and do not know the results for the testing data). A better way of doing this is **cross validation**, which means that I separate my training data in multiple data sets, parts of which I can then use for training and parts for testing. Additionally, this allows checking how stable the different parameters are (e.g. if the parameter for 'Age' has a large standard deviation, it means that is probably not playing a role..).

However, since all of this is possibly a bit annoying to do in Numpy and by hand, I will next have a look at the build-in module of sci-kit, which apparently provides this functionality.

### Linear regression using sklearn

Documentation for the sklearn linear regression module: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

Documentation for the sklearn cross_validation module: http://scikit-learn.org/stable/modules/cross_validation.html

**Import relevant modules:**



In [8]:
# Import the linear regression class
from sklearn.linear_model import LinearRegression
# Sklearn also has a helper that makes it easy to do cross validation
from sklearn.cross_validation import KFold

'LinearRegression' is a class, so I first need to initiate an instance of it:

In [9]:
# initialise LinearRegression class
LR = LinearRegression()

From the documentation on cross-validation: "In the basic approach, called k-fold CV, the training set is split into k smaller sets (other approaches are described below, but generally follow the same principles). The following procedure is followed for each of the k “folds”:

* A model is trained using k-1 of the folds as training data;
* The resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).

The performance measure reported by k-fold cross-validation is then the average of the values computed in the loop."

And from the documentation of kFold: "KFold divides all the samples in k groups of samples, called folds, of equal sizes (if possible). The prediction function is learned using k - 1 folds, and the fold left out is used for test." 
See: http://scikit-learn.org/stable/modules/cross_validation.html for how exactly it works. If I understand correclty, it produces indices which one can use to reference the training and testing set when doing the cross validation. 

The inputs of KFold are the number of rows and the number of folds that should be created. Additionally, specifying 'random_state = const.' fixes the folds so they are the same with each iteration of the code. Otherwise, folds are selected randomly.

In [42]:
# Generate cross validation folds for the titanic_train data set.
# KF returns indices to use in each iteration of the cross validation
KF = KFold(len(titanic_train),n_folds=5,random_state=1)

Next define the columns of the data set (i.e. the parameters) which I want to use for the fit.

In [110]:
# Select columns of 'titanic_train' which should be used for fitting. 
predictors = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
#predictors = ['Pclass','Sex','Age','Fare']

Now run the algorithm. The way it is done here is for educational purposes.
* First create a container for the predictions made with each of the 3 kFolds.
* Loop through the 3 kFolds and use the indices to select the respective training and testing data.
* Fit using the training data.
* Apply fit to the testing data and obtain predictions. 
* Combine all predictions and compare them to the predictions given in 'titanic_train'.

NB: I realise that the 'LinearRegression' does actually not have a function to print the coefficients obtained from the fit. These are instead 'attributes of the estimator' (I believe that is the instance of the class). However, these are also easily accessible, e.g. by printing 'LR.coef_' (type LR. and press 'tab' to get a list of attributes, or LR and 'tab' to get a list of functions).

In [111]:
# container for predictions
predictions = []
# container for fitting parameters
coefficients = []
# container for errors 
R2 = []

# loop through kFolds
for train, test in KF:
    # select rows for training and columns defined in the 'predictors' above
    train_predictors = titanic_train[predictors].iloc[train,:]
    # select rows from 'survived' column as target for training
    train_target = titanic_train['Survived'].iloc[train]
    # obtain linear fit
    LR.fit(train_predictors,train_target)
    # determine R2 of the fit
    train_R2 = LR.score(train_predictors,train_target)
    R2.append(train_R2)
    # extract coefficients of fit
    coefficients.append(LR.coef_)
    # use fit to make predictions for the test fold
    test_predictions = LR.predict(titanic_train[predictors].iloc[test,:])
    predictions.append(test_predictions)

#  concatenate 'predictions' and 'coefficients'
predictions = np.concatenate(predictions,axis=0)
coefficients = np.array(coefficients)
print (coefficients)
print (R2)

[[ -1.94770155e-01   4.83242593e-01  -5.87181818e-03  -3.01200776e-02
   -2.51506883e-02   3.99245080e-04   3.78545461e-02]
 [ -1.90435635e-01   5.04925548e-01  -5.86745877e-03  -5.44579378e-02
   -1.53140798e-02   2.95697260e-04   3.23947446e-02]
 [ -1.60392743e-01   5.05703969e-01  -6.10566228e-03  -4.25034053e-02
   -2.19101463e-02   6.09333995e-04   4.10605535e-02]
 [ -1.54380130e-01   5.26348045e-01  -6.44693377e-03  -5.31316455e-02
    1.79641915e-02   2.31350887e-04   4.58609934e-02]
 [ -1.78497200e-01   5.06138598e-01  -5.12500810e-03  -3.28888923e-02
   -2.50505591e-02   5.03408113e-05   4.00999964e-02]]
[0.40437021248719235, 0.39759260778316974, 0.40037302420560139, 0.41176701153299311, 0.38222953939333115]


'predictions' initially contained three arrays of predictions for the three kFolds of 'titanic_train'. The way it is set up, these are already in order, so concatenating them gives one column which I can directly compare to 'titanic_train['Survived']. 

The coefficients of the linear fit are relatively constant for the different kFolds, which I assume is a good sign (if they were changing a lot, it would mean they are more or less random and do not reflect any significant relation).

In the challenge the metric of success is defined as the fraction of correct predictions, so I can have a look at the training data and see how good I was doing here. Before this can be done, the predictions (which are floating point at the moment) need to be mapoped on the binary values '0' and '1'. This can be done by simply setting each value 0 if smaller than 0.5 and 1 if larger-equal 0.5.

NB: Following 'Data Science from Scratch', the parameter 'accuracy' below is actually not a very good parameter... 

In [112]:
# map predictions on binary values 0 and 1
predictions[predictions >= 0.5] = 1
predictions[predictions < 0.5] = 0

# sum the number of correct predictions and divide by total
accuracy = np.sum([predictions == titanic_train['Survived']])/len(predictions)
print (accuracy)

0.787878787879


### Creating a submission file for Kaggle

The submission file should contain two columns only: Passanger IDs of the test set and the corresponding predictions. 

We have already cleaned up the test data set, so the only thing to do is running the linear regression on it (using the parameters from the training data) and make predictions. Since we satisfied ourselfs that the model works using the kFolds before, we'll now use the full training data set for fitting and then apply the obtained relation on the test data.

In [71]:
# train algorithm on full training data set
LR.fit(titanic_train[predictors],titanic_train['Survived'])
# apply linear fit to test data set and make predictions
predictions = LR.predict(titanic_test[predictors])
# map predictions on 0 and 1
predictions[predictions >= 0.5] = 1
predictions[predictions < 0.5] = 0

# generate a submission file, containing only passanger IDs and predictions
predictions = pd.DataFrame(predictions)
submission = pd.concat([titanic_test['PassengerId'],predictions],axis=1)
submission.columns = ['PassengerId','Survived']

Finally, export the submission file:

In [72]:
submission.to_csv('Titanic_submission.csv')